In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import os

In [ ]:
# def __init__(self, root, resize, mode):
#         super(Pokemon, self).__init__()

#         self.root = root
#         self.resize = resize

#         self.name2label = {}
#         for name in sorted(os.listdir(os.path.join(root))):
#             if not os.path.isdir(os.path.join(root, name)):
#                 continue
#            # len(..) returns from 0 - n based on saved labels so far
#             self.name2label[name] = len(self.name2label.keys())

#         print(self.name2label)

#         # image, label, helper function load_csv()
#         self.images, self.labels = self.load_csv('images.csv')

#         if mode == 'train':  # 60%
#             self.images = self.images[:int(0.6 * len(self.images))]
#             self.labels = self.labels[:int(0.6 * len(self.labels))]

#         elif mode == 'val':  # 20% (60-80%)
#             self.images = self.images[int(0.6 * len(self.images)):int(0.8 * len(self.images))]
#             self.labels = self.labels[int(0.6 * len(self.labels)):int(0.8 * len(self.labels))]

#         elif mode == 'test':  # 20% (80-100%)
#             self.images = self.images[int(0.8 * len(self.images)):]
#             self.labels = self.labels[int(0.8 * len(self.labels)):]

In [ ]:
# def __getitem__(self, idx):
#         # idx: [0:len(images)]
#         # self.images, self.labels
#         # img: 'pokemon\\bulbasaur\\00000000.png'
#         # label: 0
#         img, label = self.images[idx], self.labels[idx]

#         # tf = transforms.Compose([
#         #     lambda x: Image.open(x).convert('RGB'),  # str path -> img data
#         #     transforms.Resize((int(self.resize * 1.25), int(self.resize * 1.25))),
#         #     transforms.RandomRotation(15),
#         #     transforms.CenterCrop(self.resize),
#         #     transforms.ToTensor(),
#         #     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#         #                          std=[0.229, 0.224, 0.225])  # from imagenet
#         # ])

#         img = tf(img)
#         label = torch.tensor(label)

#         return img, label

In [ ]:
# def __len__(self):
#         return len(self.images)

In [ ]:
tf = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.RandomAffine(degrees=90),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

In [ ]:
db = torchvision.datasets.ImageFolder(root='cats', transform=tf)

In [ ]:
print(db.class_to_idx)

In [ ]:
batch_size = 32
loader = torch.utils.data.DataLoader(db, batch_size=batch_size, shuffle=True, num_workers=6)
classes = ('lastik', 'other')

In [ ]:
train_set, test_set = torch.utils.data.random_split(db, [0.7, 0.3], generator=torch.Generator().manual_seed(42))
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=6)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=6)
test_len = print(len(test_set))

In [ ]:
# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


# train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                         download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
#                                           shuffle=True, num_workers=2)

# test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
#                                        download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
#                                          shuffle=False, num_workers=2)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# функция для показа изображения
def imshow(img):
    img = img  / 2  + 0.5    
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# получаем несколько случайных обучающих изображений
dataiter = iter(train_loader)
images, labels = next(dataiter)

# показать изображения
imshow(torchvision.utils.make_grid(images))
# показать лейблы изображений
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch import flatten

In [ ]:
numChannels = 3

In [ ]:
# Инициализация модели
class Net_1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # (in_channels, out_channels, kernel_size) Применяет 2D-свертку к входному сигналу, состоящему из нескольких входных плоскостей.
        self.pool = nn.MaxPool2d(2, 2) # (kernel_size, stride) Применяет MaxPool2D-объединение к входному сигналу, состоящему из нескольких входных плоскостей.
        self.conv2 = nn.Conv2d(6, 16, 5) 
        self.fc1 = nn.Linear(13456, 120)# (in_features , out_features) Применяет линейное преобразование к входящим данным
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    # Это forward функция, которая определяет структуру сети.
    # Здесь мы принимаем только один вход, но можно использовать больше.
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) #(input, start_dim) Сглаживает input путем преобразования его в одномерный тензор.
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Инициализация модели
class Net_2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=(5, 5)) # (in_channels, out_channels, kernel_size) Применяет 2D-свертку к входному сигналу, состоящему из нескольких входных плоскостей.
        self.pool = nn.MaxPool2d(2, 2) # (kernel_size, stride) Применяет MaxPool2D-объединение к входному сигналу, состоящему из нескольких входных плоскостей.
        self.conv2 = nn.Conv2d(10, 30, 5) 
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(25230, 500)# (in_features , out_features) Применяет линейное преобразование к входящим данным
        self.fc2 = nn.Linear(500, 2)

    # Это forward функция, которая определяет структуру сети.
    # Здесь мы принимаем только один вход, но можно использовать больше.
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) #(input, start_dim) Сглаживает input путем преобразования его в одномерный тензор.
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Инициализация модели
class Net_3(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 10, 5) # (in_channels, out_channels, kernel_size) Применяет 2D-свертку к входному сигналу, состоящему из нескольких входных плоскостей.
        self.pool = nn.MaxPool2d(2, 2) # (kernel_size, stride) Применяет MaxPool2D-объединение к входному сигналу, состоящему из нескольких входных плоскостей.
        self.conv2 = nn.Conv2d(10, 20, 5) 
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(20, 30, 5) 
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(4320, 120)# (in_features , out_features) Применяет линейное преобразование к входящим данным
        self.fc2 = nn.Linear(120, 80)
        self.fc3 = nn.Linear(80, 2)

    # Это forward функция, которая определяет структуру сети.
    # Здесь мы принимаем только один вход, но можно использовать больше.
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1) #(input, start_dim) Сглаживает input путем преобразования его в одномерный тензор.
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Инициализация модели
class Net_4(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 10, 5) # (in_channels, out_channels, kernel_size) Применяет 2D-свертку к входному сигналу, состоящему из нескольких входных плоскостей.
        self.pool = nn.MaxPool2d(2, 2) # (kernel_size, stride) Применяет MaxPool2D-объединение к входному сигналу, состоящему из нескольких входных плоскостей.
        self.fc1 = nn.Linear(38440, 500)# (in_features , out_features) Применяет линейное преобразование к входящим данным
        self.fc2 = nn.Linear(500, 200)
        self.fc3 = nn.Linear(200, 100)
        self.fc4 = nn.Linear(100, 2)

    # Это forward функция, которая определяет структуру сети.
    # Здесь мы принимаем только один вход, но можно использовать больше.
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = torch.flatten(x, 1) #(input, start_dim) Сглаживает input путем преобразования его в одномерный тензор.
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
nets = [] 
nets.append(Net_1())
nets.append(Net_2())
nets.append(Net_3())
nets.append(Net_4())

Коэффициент скорости обучения – это гиперпараметр, определяющий порядок того, как мы будем корректировать наши весы с учётом функции потерь в градиентном спуске. Чем ниже величина, тем медленнее мы движемся по наклонной. Хотя при использовании низкого коэффициента скорости обучения мы можем получить положительный эффект в том смысле, чтобы не пропустить ни одного локального минимума, — это также может означать, что нам придётся затратить много времени на cходимость, особенно если мы попали в область плато.

Импульс (momentum) в нейронных сетях — это вариант стохастического градиентного спуска . Он заменяет градиент импульсом , который представляет собой совокупность градиентов, как очень хорошо объяснено [здесь](https://towardsdatascience.com/10-gradient-descent-optimisation-algorithms-86989510b5e9).




In [ ]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [ ]:
import torch.optim as optim
# Критерии полезны для обучения нейронной сети. Учитывая входные данные и цель, они вычисляют градиент в соответствии с заданной функцией потерь
criterion = nn.CrossEntropyLoss()
# torch.optim - это пакет, реализующий различные алгоритмы оптимизации. Наиболее часто используемые методы уже поддерживаются, 
# а интерфейс достаточно общий, так что более сложные методы могут быть также легко интегрированы в будущем.

In [ ]:
for i, net in enumerate(nets):
    print(f'---- net {i} ----')
    print(type(net))
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(2):  # многократное прохождение по набору данных
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # получаем входные данные; данные - это список [inputs, labels].
            inputs, labels = data

            # обнуляем градиенты параметров
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # вывести статистику обучения
            running_loss += loss.item()
            if i % 10 == 9:    # вывести каждые 300 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
                running_loss = 0.0
    print('Finished Training')

    #Сохранение нашей модели
    PATH = './cats_net_'+ str(i) +'.pth'
    torch.save(net.state_dict(), PATH)

    dataiter = iter(test_loader)
    images, labels = next(dataiter)
    outputs = net(images)

    correct = 0
    total = 0
    # поскольку мы не обучаемся, нам не нужно вычислять градиенты для наших выходов
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            # рассчитываем выходные данные, пропуская изображения через сеть
            outputs = net(images)
            # класс с наибольшей мощностью - это то, что мы выбираем в качестве предсказания
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the {test_len} test images: {100 * correct // total} %')

    # Подготовка
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}

    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = net(images)
            _, predictions = torch.max(outputs, 1)
            # собираем правильные прогнозы для каждого класса
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1


    # Выводим точность на каждом классе
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

    param_count = get_n_params(net)
    print(f'Model size: {param_count} parametres')

In [ ]:
# dataiter = iter(test_loader)
# images, labels = next(dataiter)

# # вывод изображений
# imshow(torchvision.utils.make_grid(images))
# print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(2)))

In [ ]:
# net = Net_1()
# net.load_state_dict(torch.load(PATH))

In [ ]:
# outputs = net(images)

In [ ]:
# correct = 0
# total = 0
# # поскольку мы не обучаемся, нам не нужно вычислять градиенты для наших выходов
# with torch.no_grad():
#     for data in test_loader:
#         images, labels = data
#         # рассчитываем выходные данные, пропуская изображения через сеть
#         outputs = net(images)
#         # класс с наибольшей мощностью - это то, что мы выбираем в качестве предсказания
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print(f'Accuracy of the network on the {test_len} test images: {100 * correct // total} %')

In [ ]:
# # Подготовка
# correct_pred = {classname: 0 for classname in classes}
# total_pred = {classname: 0 for classname in classes}

# with torch.no_grad():
#     for data in test_loader:
#         images, labels = data
#         outputs = net(images)
#         _, predictions = torch.max(outputs, 1)
#         # собираем правильные прогнозы для каждого класса
#         for label, prediction in zip(labels, predictions):
#             if label == prediction:
#                 correct_pred[classes[label]] += 1
#             total_pred[classes[label]] += 1


# # Выводим точность на каждом классе
# for classname, correct_count in correct_pred.items():
#     accuracy = 100 * float(correct_count) / total_pred[classname]
#     print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

# Задание на практику 2
1. Модифицировать архитектуру СНС в примере (показать повышение точности)
2. Обучить свой классификатор на своем датасете
3. Назвать популярые архитектуры СНС и их особенности